In [26]:
import json
import random
import argparse
from logging import getLogger
import torch
from torch import nn
from torch.nn import functional as F

In [27]:
delete_set={'embeddings.weight','pred_layer.proj.weight','pred_layer.proj.bias'}
model_path="../dumped/arm32-x86/20240415_arm32_x86_64/periodic-1200.pth"

In [28]:
def load_model(model_path):
    model=torch.load(model_path)

    return model

### parameters in loaded model:
epoch

n_total_iter

best_metrics

best_stopping_criterion

encoder

src_decoder

tgt_decoder

dico_id2word

dico_word2id

dico_counts

params


In [29]:
def main():
    model=load_model(model_path)
    id_list = model['dico_word2id']
    UNK_id=id_list['<unk>']
    #print(f"The model is: {model}")
    # for item in model:
    #     print(item)
    tgt_embs=model['tgt_decoder']['embeddings.weight']
    src_embs=model['src_decoder']['embeddings.weight']
    shared_embs=model['encoder']['embeddings.weight']
    src_token='sub'
    src_token_id = id_list.get(src_token, UNK_id)
    src_emb= shared_embs[src_token_id]
    #print(f"src_emb:{src_emb}")

    # Calculate cosine similarity between the source token and all target embeddings
    cosine_similarities = F.cosine_similarity(src_emb.unsqueeze(0), shared_embs)

    # Sort the target tokens by their cosine similarity scores in descending order
    sorted_indices = torch.argsort(cosine_similarities, descending=True)
    id2word = {v: k for k, v in id_list.items()}

    print(f"Top target tokens similar to '{src_token}':")
    max_token_length = max(len(id2word[idx.item()]) for idx in sorted_indices)
    for idx in sorted_indices:
        tgt_token = id2word[idx.item()]
        similarity = cosine_similarities[idx].item()
        print(f"{tgt_token:{max_token_length}} {similarity:.4f}")

In [30]:
if __name__ == '__main__':
    main()

Top target tokens similar to 'sub':
sub                                            1.0000
SUBS                                           0.7655
ADD.W                                          0.5855
and                                            0.4737
BICS.W                                         0.4634
ITET                                           0.4396
VCVT.F6<VALUE>@@                               0.4384
ADDS                                           0.4384
shr                                            0.4240
cmp                                            0.4230
BCS                                            0.4103
shl                                            0.4070
jb                                             0.4013
add                                            0.3984
BVC                                            0.3977
LS@@                                           0.3974
jge                                            0.3961
LDR@@                                         

The result for all the parameters in "mlm_enfr__1024.pth"